#### Create workers dictionary
The first step in our analysis is to create the dictionary with all the workers, their level and availability.

In [2]:
def create_workers():
    main_dict = {}

    # Define the number of keys for each level
    num_keys_level_5 = 8
    num_keys_level_4 = 12
    num_keys_level_3 = 15
    num_keys_level_2 = 5

    # Populate the main dictionary with keys and corresponding dictionaries
    main_dict.update(
        {f"worker_{i}": {"level": 5, "when_will_be_available": 0} for i in range(1, num_keys_level_5 + 1)})
    main_dict.update({f"worker_{i}": {"level": 4, "when_will_be_available": 0} for i in range(
        num_keys_level_5+1, num_keys_level_5 + num_keys_level_4 + 1)})
    main_dict.update({f"worker_{i}": {"level": 3, "when_will_be_available": 0} for i in range(
        num_keys_level_5 + num_keys_level_4 + 1, num_keys_level_5 + num_keys_level_4 + num_keys_level_3 + 1)})
    main_dict.update({f"worker_{i}": {"level": 2, "when_will_be_available": 0} for i in range(num_keys_level_5 + num_keys_level_4 +
                     num_keys_level_3 + 1, num_keys_level_5 + num_keys_level_4 + num_keys_level_3 + num_keys_level_2 + 1)})

    # Print the resulting dictionary
    return main_dict


#### Get all the libraries and define the fixed costs
The next step is to get all the imports, functions and classes. Also, since it is assumed that all the workers are getting paid every month regardless of the workload, the fixed costs are defined.

In [3]:
import numpy as np
import gurobipy as gb
import seaborn as sn
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from multiprocessing import Pool, cpu_count, Process


# Costs per worker level
costs = {5: 16000, 4: 12000, 3: 10000, 2: 7000}

monthly_costs = 16000 * 8 + 12000 * 12 + 10000 * 15 + 7000 * 5
# Initial guess for the bid. necessary for scipy thing.
initial_guess = [0]

# Set bounds for optimization.
bounds = [(-np.inf, np.inf)]


#### Define the likelihood function, as well additional functions for the main analysis.


In [4]:
def likelihood_of_accepting(q, b):
    """This function firstly calculates the likelihood of project acceptance based on 
    the average quality and bid. And after that it uses np to get if the project
    was actually accepted. 

    Args:
        q (float): average quality of the team assigned to the project.
        b (float): bid per worker

    Returns:
        binary: 0 if it was simulated that the project was accepted, and 0 otherwise.
    """
    result = 1/(1 + np.exp(-6.5 - 2*q + 0.0005*b))
    outcome = np.random.choice([0, 1], p=[1-result, result])
    return outcome


def max_greedy_bid(param, people_requirement, q, d):
    """This function provides the "objection function" for greedy strategy.
    Generally speaking, this is what we are maximizing. Maybe it's possible to make it more efficient.

    Args:
        param (float): something we want to maximize.
        pple_req (integer): amount of people needed for the project.
        q (float): average quality of the team.
        d (integer): length of the project.

    Returns:
        float: the maximized bid.
    """
    b = param

    # Here we start with minus, it's not a mistake. Thew thing is that scipy has minimization function(Check later code).
    # So we basically minimizing negative outcome, aka maximizing positive one. I know it's weird, but that's how it works
    # how this library works.
    result = -b*people_requirement*d * \
        (1 / (1 + np.exp(-6.5 - 2 * q + 0.0005 * b)))
    return result

def max_greedy_bid_new(param, people_requirement, q, d):
    """This function provides the "objection function" for greedy strategy.
    Generally speaking, this is what we are maximizing. Maybe it's possible to make it more efficient.

    Args:
        param (float): something we want to maximize.
        pple_req (integer): amount of people needed for the project.
        q (float): average quality of the team.
        d (integer): length of the project.

    Returns:
        float: the maximized bid.
    """
    b = param

    # Here we start with minus, it's not a mistake. Thew thing is that scipy has minimization function(Check later code).
    # So we basically minimizing negative outcome, aka maximizing positive one. I know it's weird, but that's how it works
    # how this library works.
    result = -((b-25000)*people_requirement*d *(1 / (1 + np.exp(-6.5 - 2 * q + 0.0005 * b))))
    return result



#### Create the simulation object
It was decided to create a simulation, where number of initialization, amount of months and bid can be passed as parameters. Also, inside of this object per month simulation is performed. The outputs are functions which produce the necessary plots.

In [20]:
class Simulation:
    """Instead of creating like shit tone of similar functions, i decided to combine the whole simulation process in one 
    class. Maybe we will need to break it for later strategies, but it worked fine for strategies 1 - 4. 
    """

    def __init__(self, initialization=10000, bid=None, n=50, fixed_probability=False, greedy=False, greedy_new=False, additional_training=False, marketing = False):
        """Initiate the class.

        Args:
            bid (float, optional): If bid is fixed, than we just specify it here, or None otherwise. Defaults to None.
            n (integer, optional): Number of weeks for simulation. Defaults to 50.
            fixed_probability (bool, optional): Indicator, if it's a fixed probability approach. Defaults to False.
            greedy (bool, optional): Indicator, if it's a greedy pricing. Defaults to False.
        """
        self.initialization = initialization
        self.n = n
        self.employees = create_workers()
        self.bid = bid
        self.fixed_probability = fixed_probability
        self.greedy = greedy
        self.greedy_new = greedy_new
        self.additional_training = additional_training
        self.marketing = marketing
        if self.marketing is None:
            self.mean_for_poisson = 4
        elif self.marketing > 4000000:
            self.mean_for_poisson = 7
        elif self.marketing > 2000000:
            self.mean_for_poisson = 6
        else:
            self.mean_for_poisson = 5

        # This one means that whenever you create Simulation object it already runs the simulation.
        # And you directly can get all the values and plots.
        self._total_initialization()
        
    def _train_employees(self):
        for key, values in self.employees.items():
            if values["level"] == 2 or values["level"] == 3:
            # Update the level to 4
                values["level"] = 4
            # Keep the monthly compensation the same as their original level
                values["monthly_compensation"] = costs[values["level"]]

    def _monthly_behaviour(self, number_of_projects):
        """This is a main function. It simulates the monthly decisions - costs, revenue calculations, 
        workers allocations and others. Maybe it's smart to break it into smaller functions.  

        Args:
            num_of_proj (integer): amount of projects per month. It's the only simulation which is done on the higher level than months.

        Returns:
            tuple: revenues, costs and other proportions.
        """
        
        if self.additional_training:
            self._train_employees()
            
        # We find the sum of costs of all workers who are currently engaged in project.
        for key in self.employees.keys():
            # We are paying only to the working people, not all of them.
            if self.employees[key]["when_will_be_available"] != 0:
                # We reduce amount of "busy" periods.
                self.employees[key]["when_will_be_available"] -= 1
            
        monthly_revenue = 0
        missed_projects = 0
        taken_projects = 0
        num_of_available_workers=0
        man_months = 0

        for project in range(number_of_projects):

            # Basically, we simulate for each project the people and time requirements.
            scope_of_project = np.random.choice([3, 6], p=[0.5, 0.5])
            workers_requirement = np.random.choice(
                [3, 4, 5, 6], p=[0.2, 0.3, 0.3, 0.2])
            potential_team = []

            for worker, values in self.employees.items():
                # We checked for each worker their availability.
                # I just assume for now that better workers are checked first
                if len(potential_team) < workers_requirement:
                    if values["when_will_be_available"] == 0:
                        potential_team.append(worker)
                if values["when_will_be_available"] == 0:
                    num_of_available_workers=num_of_available_workers+1
            # Check if we managed to form a team
            if len(potential_team) == workers_requirement:

                # Find the average quality of the team. Works surprisingly fast.
                potential_quality = np.mean(
                    [self.employees[worker]["level"] for worker in potential_team])

                # It allows to hold situations, when the bid needs to be optimized.
                if not self.bid:
                    if not self.greedy:
                        if not self.greedy_new:
                            # Strategy 3
                            self.bid = 2000 * (2 * potential_quality + 6.5)
                        else:
                            # It's optimization for Strategy 4
                            if num_of_available_workers >15:
                                max_bid = minimize(max_greedy_bid, initial_guess, bounds=bounds,
                                                args=(workers_requirement, potential_quality, scope_of_project,),
                                                method='L-BFGS-B')
                                self.bid = max_bid.x[0]
                                
                            else:
                                max_bid = minimize(max_greedy_bid_new, initial_guess, bounds=bounds,
                                                args=(workers_requirement, potential_quality, scope_of_project,),
                                                method='L-BFGS-B')
                                self.bid = max_bid.x[0]
                    else:
                        max_bid = minimize(max_greedy_bid, initial_guess, bounds=bounds, args=(
                            workers_requirement, potential_quality, scope_of_project,), method='L-BFGS-B')
                        self.bid = max_bid.x[0]

                # After we got to know the bid, we find out the likelihood of acceptance and if it was actually accepted.
                is_accepted = likelihood_of_accepting(
                    q=potential_quality, b=self.bid)
                if is_accepted:
                    # If our project is actually accepted
                    # I also assume that they start working on project from the next month.
                    # That's why the calc of costs is done on the beginning of the each month.
                    for person in potential_team:
                        # Add their working months.
                        self.employees[person]["when_will_be_available"] = scope_of_project
                        man_months += scope_of_project
                    # Yeah, this one is another assumption I make, which can be changed in the future, So right now
                    # it is assumed that in month 0 we get assigned to the project and get the whole revenue for it,
                    # while starting to paying to our workers monthly from month 1 until the end.
                    monthly_revenue += self.bid * workers_requirement * scope_of_project
                    taken_projects += 1
                else:
                    # We formed the team, but project wasn't accepted.
                    potential_team = []

            elif len(potential_team) < workers_requirement:
                # If we didn't manage to form a team.
                potential_team = []
                # I add missed projects only in this case, because in the assignment he says - "he proportion of unaddressed
                # projects due to resource limitations." So we basically find, how many projects we lost only because we don't have enough people.
                missed_projects += 1
            else:
                # If we formed a team with more people in a team than needed - error.
                print("More people in a team than needed")
                break

            # If the bid is not fixed, we need to renew it for every project.
            if self.fixed_probability or self.greedy:
                self.bid = None

        return monthly_revenue, missed_projects, taken_projects, man_months

    def _one_go_intitalisation(self):
        """This is orchestration function. It runs the for loop with monthly simulations and creates all the 
        necessary metrics and values for the post simulation analysis.
        """
        # Costs and revenue are saved in lists, and not as an integer, because we will need them further for plots.
        one_go_total_revenue, one_go_total_missed_projects, one_go_total_taken_projects, one_go_total_man_months = 0, 0, 0, 0
        num_of_projects_per_month = np.random.poisson(self.mean_for_poisson, size=self.n)
        # This is monthly simulation creation.
        for monthly_proj in num_of_projects_per_month:
            monthly_revenue, missed_projects, taken_projects, man_months = self._monthly_behaviour(
                monthly_proj)
            one_go_total_revenue += monthly_revenue
            one_go_total_missed_projects += missed_projects
            one_go_total_taken_projects += taken_projects
            one_go_total_man_months += man_months

        one_go_total_costs = monthly_costs * self.n
        total_num_of_projects = num_of_projects_per_month.sum()

        # Create the total profit list.
        one_go_total_profit = one_go_total_revenue - one_go_total_costs

        # Missed proj/all the projects. I calulate the total like this,
        # because the proportion missed_proj/taken_proj is more than 1.
        one_go_proportion_of_missed_proj = one_go_total_missed_projects / \
            (total_num_of_projects)

        # Utilization of man-months out of available 40 people * 50 months.
        one_go_utilization_man_months = one_go_total_man_months/(40 * 50)

        return one_go_total_revenue, one_go_total_profit, one_go_proportion_of_missed_proj, one_go_utilization_man_months, total_num_of_projects

    def _total_initialization(self):

        self.total_revenue, self.total_profit, self.proportion_of_missed_proj, self.total_utilization, self.total_projects = [], [], [], [], []

        for initialization in range(self.initialization + 1):
            # print(initialization)
            one_go_total_revenue, one_go_total_profit, one_go_proportion_of_missed_proj, one_go_utilization_man_months, sum_projects = self._one_go_intitalisation()
            self.total_revenue.append(one_go_total_revenue)
            self.total_profit.append(one_go_total_profit)
            self.proportion_of_missed_proj.append(
                one_go_proportion_of_missed_proj)
            self.total_utilization.append(one_go_utilization_man_months)
            self.total_projects.append(sum_projects)

    def revenue_plot(self):
        """Creates revenue histogram.
        """
        ax = sn.histplot(self.total_revenue, kde=True, bins=50)
        ax.set(xlabel="Revenue", ylabel="Probability")
        plt.show()
        # Never forget to close the plot.
        # Otherwise each plt.show() will produce the same plot.
        plt.close()

    def profit_plot(self):
        """Creates profit histogram.
        """
        a2 = sn.histplot(self.total_profit, kde=True, bins=50)
        a2.set(xlabel="Profit", ylabel="Probability")
        plt.show()
        plt.close()

    def missed_project_proportion_plot(self):
        """Creates profit histogram.
        """
        a2 = sn.histplot(self.proportion_of_missed_proj, kde=True, bins=50)
        a2.set(xlabel="Proportion of missed project", ylabel="Probability")
        plt.show()
        plt.close()

    def man_month_utilization_plot(self):
        """Creates profit histogram.
        """
        a2 = sn.histplot(self.total_utilization, kde=True, bins=50)
        a2.set(xlabel="Man month utilization", ylabel="Probability")
        plt.show()
        plt.close()

#### Strategy 1

In [ ]:
strategy_1 = Simulation(bid=20000)
strategy_1.revenue_plot()
strategy_1.profit_plot()
strategy_1.missed_project_proportion_plot()
strategy_1.man_month_utilization_plot()
np.mean(strategy_1.total_revenue)

#### Strategy 2

In [ ]:
proposed_bids = np.random.randint(15000, 40000, 10)
highest_expected_revenue = 0
# highest_expected_revenue = 39959221.68823118
best_bid = 0
# best bid = 24647
outcome = {}
for bid in proposed_bids:
    print(f"Current bid {bid}")
    strategy = Simulation(bid=bid)
    strategy.revenue_plot()
    strategy.profit_plot()
    strategy.missed_project_proportion_plot()
    strategy.man_month_utilization_plot()
    if np.mean(strategy.total_revenue) > highest_expected_revenue:
        best_bid = bid
        highest_expected_revenue = np.mean(strategy.total_revenue)
    outcome[bid] = np.mean(strategy.total_revenue)

#### Strategy 3

In [ ]:
strategy_3 = Simulation(initialization=100, fixed_probability=True)
strategy_3.revenue_plot()
strategy_3.profit_plot()
strategy_3.missed_project_proportion_plot()
strategy_3.man_month_utilization_plot()
np.mean(strategy_3.total_revenue)
# Exp rev = 42943152.48475152

#### Strategy 4

In [ ]:
strategy_4 = Simulation(n=50, greedy=True)
strategy_4.revenue_plot()
strategy_4.profit_plot()
strategy_4.missed_project_proportion_plot()
strategy_4.man_month_utilization_plot()
np.mean(strategy_4.total_revenue)

#### Strategy 5

In [ ]:
strategy_5 = Simulation(greedy_new=True)
strategy_5.revenue_plot()
strategy_5.profit_plot()
strategy_5.missed_project_proportion_plot()
strategy_5.man_month_utilization_plot()

#### Strategy 6

In [ ]:
strategy_6 = Simulation(bid=20000, additional_training=True)
total_revenue_strategy_6 = np.mean(strategy_6.total_revenue)
total_profit_strategy_6 = np.mean(strategy_6.total_profit)
proportion_of_missed_proj_strategy_6 = np.mean(strategy_6.proportion_of_missed_proj)
total_utilization_strategy_6 = np.mean(strategy_6.total_utilization)
total_projects_6 = np.sum(strategy_6.total_projects)
strategy_6.revenue_plot()
strategy_6.profit_plot()
strategy_6.missed_project_proportion_plot()
strategy_6.man_month_utilization_plot()

#### Strategy 7.1 (2 mil invested)

In [ ]:
strategy_7_1 = Simulation(bid=20000, marketing = 2000000)

total_revenue_strategy_7_1 = np.mean(strategy_7_1.total_revenue)
total_profit_strategy_7_1 = np.mean(strategy_7_1.total_profit)
proportion_of_missed_proj_strategy_7_1 = np.mean(strategy_7_1.proportion_of_missed_proj)
total_utilization_strategy_7_1 = np.mean(strategy_7_1.total_utilization)
total_projects_7_1 = np.sum(strategy_7_1.total_projects)

strategy_7_1.revenue_plot()
strategy_7_1.profit_plot()
strategy_7_1.missed_project_proportion_plot()
strategy_7_1.man_month_utilization_plot()

#### Strategy 7.2 (4 mil invested)

In [ ]:
strategy_7_2 = Simulation(bid=20000, marketing = 4000000)

total_revenue_strategy_7_2 = np.mean(strategy_7_2.total_revenue)
total_profit_strategy_7_2 = np.mean(strategy_7_2.total_profit)
proportion_of_missed_proj_strategy_7_2 = np.mean(strategy_7_2.proportion_of_missed_proj)
total_utilization_strategy_7_2 = np.mean(strategy_7_2.total_utilization)
total_projects_7_2 = np.sum(strategy_7_2.total_projects)

strategy_7_2.revenue_plot()
strategy_7_2.profit_plot()
strategy_7_2.missed_project_proportion_plot()
strategy_7_2.man_month_utilization_plot()